In [19]:
import progNotebook
import datasource
import BinnedScat
import json
import numpy as np
from fast_histogram import histogram1d

In [20]:
widget = progNotebook.ProgScatterWidget()
display(widget)

ProgScatterWidget()

Restart changed!!!!!! restart 0 1
Restart changed!!!!!! restart 0 1


In [44]:
def on_value_change(change):
    print('UWOWOWUWOW',change['new'])
widget.observe(on_value_change, names='restart')

In [8]:
widget.restart = 13

Restart changed!!!!!! restart 10 13


In [21]:
widget.restart = 0

Restart changed!!!!!! restart 1 0


In [7]:
def processCountChunk(chunk,progress):
    scat.updateWithDataFrame(chunk, 'B', 'C', 'A')
    valueBB = scat.getAggregationRange()
    summary = scat.getSummary()
    # build data to pass to widget
    if summary:
        bbox = scat.getFixedBoundingBox()
        scatData = {'xDomainRange': bbox[0],
                    'yDomainRange': bbox[1],
                    'zDomainRange': valueBB,
                    'xRes': scat._resX,
                    'yRes': scat._resY,
                    'xLabel': 'xAxis',
                    'yLabel': 'yAxis',
                    'zLabel': 'zLabel',
                    'sparse': 0,
                    'bins': summary[0].tolist(),
                    'counts': summary[1].tolist()
                    }

        #
        histResolution = 10
        pointsX = scat.getMarginalHistogram(BinnedScat.Axis.X,histResolution)
        pointsX = [[pointsX[0][i],pointsX[1][i]] for i in range(len(pointsX[0]))]
        #
        pointsY = scat.getMarginalHistogram(BinnedScat.Axis.Y,histResolution)
        pointsY = [[pointsY[0][i],pointsY[1][i]] for i in range(len(pointsY[0]))]
        #
        zHist = scat.getCountHistogram()
        numSplits = 10 + 1
        xMin = zHist['range'][0]
        xMax = zHist['range'][1]
        step = (xMax - xMin) / numSplits
        splits = [xMin + (i*step) for i in range(0, numSplits)]
        pointsZ = list(zip(splits,list(zHist['values'])))
        #
        mx = {'points': pointsX, 'xLabel': 'Marginal X'}        
        my = {'points': pointsY, 'xLabel': 'Marginal Y'}
        mz = {'points': pointsZ, 'xLabel': 'Marginal Z'}
        obj = {"mx":mx,"my":my,"mz":mz}
        myProgress = {'current': progress[0], 'max': progress[1]}
        #
        try:
            widget.marginals = json.dumps(obj)
            widget.scatData = json.dumps(scatData)
            widget.progress = json.dumps(myProgress)
        except:
            print(scat.getSummary())
            halt
        #print('*******',progress)
       
scat = BinnedScat.CountBinScat('B', 'C', 'A', 10, 10, 20)
datasource.DataSource("data/sample.csv",processCountChunk,10).start()
#datasource.DataSource("data/bigfile_multiscale.csv",processCountChunk,10000).start()

{'points': [[-2.4325320720672607, 0.0], [-0.9508976936340332, 0.0], [0.5307366847991943, 0.0132], [2.012371063232422, 0.0569], [3.4940054416656494, 0.1565], [4.975639820098877, 0.2627], [6.4572741985321045, 0.2725], [7.938908576965332, 0.1649], [9.42054295539856, 0.0589], [10.902177333831787, 0.0096], [12.383811712265015, 0.0048]], 'xLabel': 'Marginal X'}
{'points': [[-3.500880479812622, 0.0], [-1.9221235188570889, 0.0016], [-0.3433665579015557, 0.0042], [1.2353904030539775, 0.0257], [2.8141473640095107, 0.113], [4.392904324965044, 0.2368], [5.971661285920577, 0.3091], [7.550418246876109, 0.2167], [9.129175207831643, 0.0711], [10.707932168787178, 0.0214], [12.28668912974271, 0.0004]], 'xLabel': 'Marginal X'}
{'points': [[-3.500880479812622, 0.0], [-1.9221235188570889, 0.0005333333333333334], [-0.3433665579015557, 0.004666666666666667], [1.2353904030539775, 0.027533333333333333], [2.8141473640095107, 0.10886666666666667], [4.392904324965044, 0.23983333333333334], [5.971661285920577, 0.3

{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.00030476190476190474], [0.23375857960094137, 0.009180952380952382], [1.9377195184881035, 0.056933333333333336], [3.6416804573752657, 0.18295714285714285], [5.34564139626243, 0.31689047619047617], [7.049602335149592, 0.2796904761904762], [8.753563274036754, 0.12683333333333333], [10.457524212923916, 0.026904761904761904], [12.161485151811078, 0.00030476190476190474]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.0002909090909090909], [0.23375857960094137, 0.00909090909090909], [1.9377195184881035, 0.05715], [3.6416804573752657, 0.1823], [5.34564139626243, 0.31765], [7.049602335149592, 0.27941818181818184], [8.753563274036754, 0.12729545454545454], [10.457524212923916, 0.026218181818181818], [12.161485151811078, 0.0005863636363636364]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1

{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 8e-05], [0.23375857960094137, 0.010605], [1.9377195184881035, 0.052065], [3.6416804573752657, 0.18452], [5.34564139626243, 0.32312], [7.049602335149592, 0.2760175], [8.753563274036754, 0.1250625], [10.457524212923916, 0.02789], [12.161485151811078, 0.00064]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 7.804878048780487e-05], [0.23375857960094137, 0.010936585365853659], [1.9377195184881035, 0.052034146341463416], [3.6416804573752657, 0.18450731707317072], [5.34564139626243, 0.32298536585365856], [7.049602335149592, 0.2765878048780488], [8.753563274036754, 0.12467560975609757], [10.457524212923916, 0.0279609756097561], [12.161485151811078, 0.00023414634146341464]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.0], [0.23375857960094137, 0.010971428571428571], [1.93

{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.0004689655172413793], [0.23375857960094137, 0.011782758620689655], [1.9377195184881035, 0.048174137931034484], [3.6416804573752657, 0.18719655172413793], [5.34564139626243, 0.3195913793103448], [7.049602335149592, 0.2787396551724138], [8.753563274036754, 0.12477241379310344], [10.457524212923916, 0.026568965517241378], [12.161485151811078, 0.0027051724137931036]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.00043389830508474577], [0.23375857960094137, 0.012050847457627118], [1.9377195184881035, 0.048110169491525426], [3.6416804573752657, 0.18726610169491525], [5.34564139626243, 0.31960677966101697], [7.049602335149592, 0.27868474576271185], [8.753563274036754, 0.12488474576271187], [10.457524212923916, 0.026359322033898305], [12.161485151811078, 0.0026033898305084747]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 

{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.0003368421052631579], [0.23375857960094137, 0.011031578947368422], [1.9377195184881035, 0.04798421052631579], [3.6416804573752657, 0.1884578947368421], [5.34564139626243, 0.3208421052631579], [7.049602335149592, 0.2773473684210526], [8.753563274036754, 0.12397631578947368], [10.457524212923916, 0.025644736842105262], [12.161485151811078, 0.004378947368421053]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.1741632981733843, 0.0], [-1.4702023592862217, 0.0], [0.23375857960094137, 0.011194805194805195], [1.9377195184881035, 0.048405194805194805], [3.6416804573752657, 0.1879909090909091], [5.34564139626243, 0.32113506493506494], [7.049602335149592, 0.27755844155844156], [8.753563274036754, 0.12362597402597403], [10.457524212923916, 0.025435064935064935], [12.161485151811078, 0.004654545454545455]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.17416329817338

{'points': [[-4.878124237060547, 0.0], [-3.0961076563054863, 0.0], [-1.3140910755504263, 0.0005389473684210526], [0.46792550520463383, 0.011877894736842105], [2.2499420859596944, 0.07018], [4.031958666714754, 0.22986736842105263], [5.8139752474698145, 0.34502947368421055], [7.595991828224875, 0.24895052631578948], [9.378008408979936, 0.08169894736842105], [11.160024989734996, 0.011856842105263159], [12.942041570490055, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.0961076563054863, 0.0], [-1.3140910755504263, 0.0005333333333333334], [0.46792550520463383, 0.011879166666666666], [2.2499420859596944, 0.07049270833333333], [4.031958666714754, 0.23008020833333334], [5.8139752474698145, 0.344984375], [7.595991828224875, 0.248403125], [9.378008408979936, 0.08162708333333334], [11.160024989734996, 0.012], [12.942041570490055, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.0961076563054863, 0.0], [-1.3140910755504263, 0.0010556701030927835]

{'points': [[-4.878124237060547, 0.0], [-3.0778042186390273, 0.0], [-1.2774842002175073, 0.0009061946902654867], [0.5228358182040127, 0.014614159292035397], [2.3231558366255323, 0.07236106194690266], [4.123475855047053, 0.2423], [5.923795873468572, 0.34653451327433626], [7.724115891890092, 0.23597522123893805], [9.524435910311611, 0.07405575221238939], [11.324755928733133, 0.013253097345132744], [13.125075947154652, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.0778042186390273, 0.0], [-1.2774842002175073, 0.0008982456140350877], [0.5228358182040127, 0.014433333333333333], [2.3231558366255323, 0.07213333333333333], [4.123475855047053, 0.24240175438596492], [5.923795873468572, 0.3461315789473684], [7.724115891890092, 0.23630350877192982], [9.524435910311611, 0.0740561403508772], [11.324755928733133, 0.013642105263157895], [13.125075947154652, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.0778042186390273, 0.0], [-1.2774842002175073

{'points': [[-4.878124237060547, 0.0], [-3.0778042186390273, 0.0], [-1.2774842002175073, 0.0007757575757575758], [0.5228358182040127, 0.012896969696969697], [2.3231558366255323, 0.07261515151515152], [4.123475855047053, 0.24282272727272727], [5.923795873468572, 0.347369696969697], [7.724115891890092, 0.23583030303030303], [9.524435910311611, 0.07410681818181818], [11.324755928733133, 0.013582575757575757], [13.125075947154652, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.0778042186390273, 0.0], [-1.2774842002175073, 0.0007699248120300752], [0.5228358182040127, 0.01290827067669173], [2.3231558366255323, 0.07287142857142857], [4.123475855047053, 0.24258496240601504], [5.923795873468572, 0.3474511278195489], [7.724115891890092, 0.2356984962406015], [9.524435910311611, 0.07423759398496241], [11.324755928733133, 0.013478195488721804], [13.125075947154652, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-3.0778042186390273, 0.0], [-1.277484

{'points': [[-4.878124237060547, 0.0], [-2.9661095359108662, 0.0], [-1.054094834761186, 0.0017066666666666667], [0.8579198663884942, 0.019216], [2.769934567538175, 0.11075466666666667], [4.681949268687855, 0.30458466666666667], [6.593963969837535, 0.35029733333333335], [8.505978670987217, 0.17201066666666667], [10.417993372136896, 0.03596866666666667], [12.330008073286578, 0.005461333333333334], [14.242022774436258, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-2.9661095359108662, 0.0], [-1.054094834761186, 0.001695364238410596], [0.8579198663884942, 0.019210596026490067], [2.769934567538175, 0.1107860927152318], [4.681949268687855, 0.3043682119205298], [6.593963969837535, 0.35043245033112586], [8.505978670987217, 0.17204900662251654], [10.417993372136896, 0.036033112582781454], [12.330008073286578, 0.005425165562913907], [14.242022774436258, 0.0]], 'xLabel': 'Marginal X'}
{'points': [[-4.878124237060547, 0.0], [-2.9661095359108662, 0.0], [-1.054094834761186, 

KeyboardInterrupt: 

In [ ]:
def processMeanChunk(chunk):
    scat.updateWithDataFrame(chunk, 'B', 'C', 'A')
    valueBB = scat.getAggregationRange()
    summary = scat.getSummary()
    # build data to pass to widget
    if summary:
        bbox = scat.getFixedBoundingBox()
        scatData = {'xDomainRange': bbox[0],
                    'yDomainRange': bbox[1],
                    'zDomainRange': valueBB,
                    'xRes': scat._resX,
                    'yRes': scat._resY,
                    'xLabel': 'xAxis',
                    'yLabel': 'yAxis',
                    'zLabel': 'zLabel',
                    'sparse': 0,
                    'bins': summary[0],
                    'counts': summary[1]
                    }

        #
        histResolution = 10
        pointsX = scat.getMarginalHistogram(BinnedScat.Axis.X,histResolution)
        pointsX = [[pointsX[0][i],pointsX[1][i]] for i in range(len(pointsX[0]))]
        #
        pointsY = scat.getMarginalHistogram(BinnedScat.Axis.Y,histResolution)
        pointsY = [[pointsY[0][i],pointsY[1][i]] for i in range(len(pointsY[0]))]
        #
        pointsZ = scat.getMarginalHistogram(BinnedScat.Axis.Z,histResolution)
        pointsZ = [[pointsY[0][i],pointsY[1][i]] for i in range(len(pointsY[0]))]
        #
        mx = {'points': pointsX, 'xLabel': 'Marginal X'}
        my = {'points': pointsY, 'xLabel': 'Marginal Y'}
        mz = {'points': pointsZ, 'xLabel': 'Marginal Z'}
        obj = {"mx":mx,"my":my,"mz":mz}
        try:
            widget.marginals = json.dumps(obj)
            #json.dumps(scatData)
            widget.scatData = json.dumps(scatData)
        except:
            print(summary)
            halt
       
scat = BinnedScat.MeanBinScat('B', 'C', 'A', 10, 10, 20)
datasource.DataSource("data/sample.csv",processMeanChunk,10).start()
#datasource.DataSource("data/bigfile_multiscale.csv",processChunk,1000).start()